# Flask API to generate text with ChatGPT

In [ ]:
## Import libraries and load env vars
import openai
from openai import AsyncOpenAI, OpenAI
from flask import Flask, request, jsonify
import os
from dotenv import load_dotenv
from flask_jwt_extended import JWTManager, create_access_token, jwt_required, get_jwt_identity
import os

load_dotenv()

In [ ]:
# Read the prompts from JSON file
import json

with open(os.getenv("PROMPT_FILE_PATH"), 'r') as file:
    prompts = json.load(file)


## Create a API Endpoint 
### Request example

```json
{
    "prompt": "We are going to create a page about a product to try to sell it on an affiliate market and make a profit. Generate 2 paragraphs (about 1000 words each of them) for this product based on the following keywords, the first one is the product: ",
    "keywords": "'Chaqueta de cuero', 'Rock', 'ropa', 'chupa de cuero', 'calidad', 'precio'",
    "system-role": "Act as you are a helpful assistant for SEO purposes. You will only give the content of the requested text, without intros or outros. You will speak only in ESPAÑOL with a SPAIN PEOPLE accent."
}
```


In [ ]:
client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"), project=os.getenv("OPENAI_PROJECT_ID"))

app = Flask(__name__)

# Config JWT
app.config['JWT_SECRET_KEY'] = os.getenv('JWT_SECRET_KEY')
app.config['JWT_ACCESS_TOKEN_EXPIRES'] = int(os.getenv('JWT_ACCESS_TOKEN_EXPIRES'))
jwt = JWTManager(app)

# Auth endpoint
@app.route('/login', methods=['POST'])
def login():
    username = request.json.get("username", None)
    password = request.json.get("password", None)

    # Check username and password
    if username != "testuser" or password != "testpassword":
        return jsonify({"msg": "Bad username or password"}), 401

    # Crea un nuevo token de acceso
    access_token = create_access_token(identity=username)
    return jsonify(access_token=access_token)

# Text generation endpoint
@app.route('/generate-text', methods=['POST'])
@jwt_required()
def generate_text():
    try:
        response = client.chat.completions.create(
            model=os.getenv("OPENAI_MODEL", "gpt-3.5-turbo"),
            messages=[
                {"role": "system", "content": request.json["system-role"]},
                {"role": "user", "content": request.json["prompt"] + " " + request.json["keywords"]},
            ],
            # max_tokens=int(os.getenv("OPENAI_MAX_TOKENS")),
            n=1,
            stop=None,
            temperature=float(os.getenv("OPENAI_TEMPERATURE")),
        )
        generated_text = response.choices[0].message.content.strip()

        return jsonify({'text': generated_text, 'prompt': request.json["prompt"]})
    except Exception as e:
        return jsonify({'error': str(e)}), 500

In [ ]:
if __name__ == '__main__':
    app.run(host=os.getenv("API_HOST", "0.0.0.0"), port=int(os.getenv("API_PORT", 5000)))